In [7]:
%cd /userspace/pva/nsu-ai-taiga_version
%pwd

/userspace/pva/nsu-ai-taiga_version


In [9]:
from team_code.generate import setup_model_and_tokenizer

/userspace/pva/miniconda3/envs/py9min/lib/python3.9/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Using default config


2024-07-21 11:41:03 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2024-07-21 11:41:03 | INFO | team_code.ONE_PEACE.one_peace.models.components | ****** Import flash_attn.ops.layer_norm fail, please install flash_attn ******
2024-07-21 11:41:03 | INFO | team_code.ONE_PEACE.one_peace.models.transformer.multihead_attention | ****** Import memory_efficient_attention fail, please install xFormers ******
2024-07-21 11:41:03 | INFO | team_code.ONE_PEACE.one_peace.models.one_peace.one_peace_base | ****** Import memory_efficient_attention fail, please install xFormers ******
2024-07-21 11:41:04 | WARNING | matplotlib | Matplotlib created a temporary cache directory at /tmp/matplotlib-9d1tvode because the default path (/home/pva/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multip

WARNING ⚠️ user config directory '/home/pva/.config/Ultralytics' is not writeable, defaulting to '/tmp' or CWD.Alternatively you can define a YOLO_CONFIG_DIR environment variable for this path.


In [10]:
model, processor = setup_model_and_tokenizer()

2024-07-21 11:41:13 | INFO | team_code.generate | Current working directory: /userspace/pva/nsu-ai-taiga_version
2024-07-21 11:41:13 | INFO | team_code.generate | New working directory: /userspace/pva/nsu-ai-taiga_version/team_code/ONE-PEACE
2024-07-21 11:41:16 | INFO | team_code.ONE_PEACE.one_peace.tasks.base_task | dictionary: 50264 types
/userspace/pva/miniconda3/envs/py9min/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
2024-07-21 11:42:16 | INFO | team_code.ONE_PEACE.one_peace.models.one_peace.one_peace_retrieval | logit_scale not exists, re-initialized
2024-07-21 11:42:21 | INFO | team_code.generate | ONE-PEACE model is loaded from the "/userspace/dra/nsu-ai/team_code/models/one-peace.pt".
2024-07-21 11:42:21 | INFO | team_code.gen

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2024-07-21 11:43:26 | INFO | team_code.generate | The large language model is loaded.
2024-07-21 11:43:46 | INFO | team_code.generate | The Ocr model is loaded.
2024-07-21 11:43:46 | INFO | team_code.generate | The YOLOv8 model is loaded.
/userspace/pva/miniconda3/envs/py9min/lib/python3.9/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
2024-07-21 11:43:48 | INFO | team_code.generate | The Translation models are loaded.


In [11]:
from team_code.generate import setup_model_and_tokenizer, load_images, MultimodalModel, DEVICE
import json
from peft import LoraConfig, get_peft_model
from transformers import LlavaNextProcessor
from torch.utils.data import Dataset
import torch
from transformers import Trainer, TrainingArguments
import sys

In [12]:
class Llava_finetuning_Dataset(Dataset):
    def __init__(self, json_file: str, processor: LlavaNextProcessor):
        with open(json_file, 'r') as f:
            self.data = json.load(f)
        self.processor = processor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        item = self.data[idx]
        return {'text': item['conversations'][0]['value'], 'image': item['image'], 'answer': item['conversations'][1]['value']}

In [13]:

class Collator:
    def __init__(self, processor):
        self.processor = processor

    def __call__(self, samples):

        images = [item['image'] for item in samples]
        human_texts = [item['text'] for item in samples]
        gpt_texts = [item['answer'] for item in samples]

        loaded_images= load_images(images)
        
        p_inputs = []
        for prompt, image in zip(human_texts, loaded_images):
            p_inputs.append(self.processor(text=prompt, images=image, return_tensors="pt"))

        return {'input': p_inputs, 'label': gpt_texts}

In [14]:
def find_all_linear_names(model):
    """
    :param model: модель
    :return: list: список линейных слоев, оставленных для тренировки
    """
    cls = torch.nn.Linear  # Use the standard Linear class for LLaVA-NeXT
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
        if 'lm_head' in lora_module_names:  # Remove 'lm_head' if it's not applicable for LLaVA-NeXT
            lora_module_names.remove('lm_head')
    return list(lora_module_names)

In [ ]:
def train(model: MultimodalModel, processor: LlavaNextProcessor, batch_size: int, epochs: int, dir_train_dataset='train_dataset.json', dir_eval_dataset='eval_dataset.json'):
    

    modules = find_all_linear_names(model.llm) 
    train_dataset = Llava_finetuning_Dataset(json_file=dir_train_dataset, processor=processor)

    eval_dataset = Llava_finetuning_Dataset(json_file=dir_eval_dataset, processor=processor)
    collator = Collator(processor)

    # Configuration
    lora_config = LoraConfig( 
        r=8,
        lora_alpha=32,
        target_modules=modules,
        bias="none",
        task_type="CAUSAL_LM"
    )

    llm_model = get_peft_model(model.llm, lora_config)
    llm_model


    training_args = TrainingArguments(
        output_dir="/userspace/pva/train/results",
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir="/userspace/pva/train/logs",
    )

    # Initialize the Trainer
    trainer = Trainer(
        model=llm_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=collator,
    )
    # Train the model
    trainer.train()

: 

In [ ]:
train_json_path, eval_dataset = "/userspace/pva/train/train.json", "/userspace/pva/train/validation.json"
train(model, processor, 16, 10, train_json_path, eval_dataset)

2024-07-21 11:48:42 | INFO | peft.tuners.tuners_utils | Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!


ValueError: Target module Dropout(p=0.05, inplace=False) is not supported. Currently, only the following modules are supported: `torch.nn.Linear`, `torch.nn.Embedding`, `torch.nn.Conv2d`, `transformers.pytorch_utils.Conv1D`.